In [ ]:
import json
import csv
import os
import requests
from bs4 import BeautifulSoup
import aiohttp
import asyncio
import nest_asyncio

# Allow asyncio to run in environments that already have an event loop running
nest_asyncio.apply()

class WeatherData:
    def __init__(self, city_name, temperature, humidity, wind_speed, description):
        self.city_name = city_name
        self.temperature = temperature
        self.humidity = humidity
        self.wind_speed = wind_speed
        self.description = description

    def to_dict(self):
        return {
            "city_name": self.city_name,
            "temperature": self.temperature,
            "humidity": self.humidity,
            "wind_speed": self.wind_speed,
            "description": self.description
        }

class WeatherDataHandler:
    def __init__(self, save_directory="weather_data"):
        self.save_directory = save_directory
        try:
            os.makedirs(self.save_directory, exist_ok=True)
        except PermissionError as e:
            print(f"Failed to create directory {self.save_directory}: {e}")
            raise

    def save_json(self, data, filename):
        """
        Save data to a JSON file.

        :param data: Data to save (should be serializable to JSON).
        :param filename: Name of the file to save the data in.
        """
        filepath = os.path.join(self.save_directory, f"{filename}.json")
        try:
            with open(filepath, 'w', encoding='utf-8') as json_file:
                json.dump(data, json_file, indent=4)
            print(f"Data successfully saved to {filepath}")
        except Exception as e:
            print(f"Failed to save data to JSON: {e}")

    def save_csv(self, data, filename, fieldnames):
        """
        Save data to a CSV file.

        :param data: List of dictionaries containing the data to save.
        :param filename: Name of the file to save the data in.
        :param fieldnames: List of field names for the CSV header.
        """
        filepath = os.path.join(self.save_directory, f"{filename}.csv")
        try:
            with open(filepath, mode='w', newline='', encoding='utf-8') as csv_file:
                writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
                writer.writeheader()
                for row in data:
                    writer.writerow(row)
            print(f"Data successfully saved to {filepath}")
        except Exception as e:
            print(f"Failed to save data to CSV: {e}")

    def save_txt(self, data, filename):
        """
        Save data to a TXT file.

        :param data: Data to save (should be a string or converted to string).
        :param filename: Name of the file to save the data in.
        """
        filepath = os.path.join(self.save_directory, f"{filename}.txt")
        try:
            with open(filepath, 'w', encoding='utf-8') as txt_file:
                txt_file.write(data)
            print(f"Data successfully saved to {filepath}")
        except Exception as e:
            print(f"Failed to save data to TXT: {e}")

    async def fetch_weather_data_async(self, cities):
        """
        Fetch weather data from multiple sources asynchronously.

        :param cities: List of cities to get the weather data for.
        """
        async with aiohttp.ClientSession() as session:
            tasks = [self.fetch_weather_from_open_meteo(session, city) for city in cities]
            await asyncio.gather(*tasks)

    async def fetch_weather_from_open_meteo(self, session, city):
        """
        Fetch weather data from Open-Meteo API for a specific city.

        :param session: Aiohttp session.
        :param city: City name to get the weather data for.
        """
        url = f"https://api.open-meteo.com/v1/forecast?latitude=35.6895&longitude=139.6917&current_weather=true"
        try:
            print(f"Fetching weather data for {city} from Open-Meteo API")
            async with session.get(url) as response:
                if response.status == 200:
                    data = await response.json()
                    weather_data = WeatherData(
                        city_name=city,
                        temperature=data["current_weather"]["temperature"],
                        humidity="N/A",  # Open-Meteo might not provide humidity
                        wind_speed=data["current_weather"]["windspeed"],
                        description="Current weather data"
                    )
                    # Save JSON, CSV, and TXT
                    self.save_json(weather_data.to_dict(), f"{city}_open_meteo_weather_data")
                    fieldnames = ["city_name", "temperature", "humidity", "wind_speed", "description"]
                    self.save_csv([weather_data.to_dict()], f"{city}_open_meteo_weather_data", fieldnames)
                    txt_data = f"City: {weather_data.city_name}\nTemperature: {weather_data.temperature}°C\nWind Speed: {weather_data.wind_speed} m/s\nDescription: {weather_data.description}\n"
                    self.save_txt(txt_data, f"{city}_open_meteo_weather_data_summary")
                else:
                    print(f"Failed to fetch data for {city} from Open-Meteo: Status {response.status}")
        except Exception as e:
            print(f"Failed to fetch data for {city} from Open-Meteo: {e}")

# Example usage
if __name__ == "__main__":
    # Creating an instance of WeatherDataHandler
    handler = WeatherDataHandler(save_directory=os.path.expanduser("~/Documents/weather_data"))

    # Fetching weather data for multiple cities asynchronously
    cities = ["London", "Paris", "Berlin", "Tokyo", "New York"]
    asyncio.run(handler.fetch_weather_data_async(cities))
